In [1]:
!pip install pyspark==2.4.5

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, StringType

from ibm_botocore.client import Config
import ibm_boto3
import os
from pyspark.sql.functions import unix_timestamp, to_date
import pyspark.sql.functions as F


import numpy as np
import pandas as pd
import json
import pyspark.sql
from pyspark.sql.functions import col, skewness, kurtosis
from pyspark.sql.functions import isnan, when, count, col
from pyspark.sql.functions import when
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType
from datetime import date, timedelta, datetime
import time

sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .getOrCreate()

Hidden cell with credentials for accessing from Cloud Object Storage

In [ ]:
# The code was removed by Watson Studio for sharing.

In [3]:
from zipfile import ZipFile

cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
cos.download_file(Bucket=credentials['BUCKET'],Key=credentials['FILE'][0],Filename=credentials['FILE'][0])
cos.download_file(Bucket=credentials['BUCKET'],Key=credentials['FILE'][1],Filename=credentials['FILE'][1])
cos.download_file(Bucket=credentials['BUCKET'],Key=credentials['FILE'][2],Filename=credentials['FILE'][2])

with ZipFile(credentials['FILE'][0],'r') as zipObj:
    zipObj.extractall()

In [6]:
spark.read.text("movies.dat").createOrReplaceTempView("movies_view")

## To use spark.sql, it should be at least a temporary view or even an table
spark.sql(""" select
split(value,'::')[0] as movieid,
split(value,'::')[1] as moviename,
substring(split(value,'::')[1],length(split(value,'::')[1])-4,4) as year
from movies_view """).createOrReplaceTempView("movies");

## To view the records, use spark.sql("select * from movies").show()
result=spark.sql("Select * from movies m1 where m1.year=(Select min(m2.year) from movies m2)").repartition(1).rdd
#to check: print(result.collect())

##To save it on the distributed file system on cloud
result.saveAsTextFile("file:///home/username/result")

In [20]:
## To save it locally
from IPython.display import HTML 
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "result.csv"):  
    csv = df.to_csv(sep=',',header=True, index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)


# rdd to df
result_df = result.toDF().toPandas()
create_download_link(result_df,filename='oldest_movies.csv')